| Register Name                | Interface       | Slave Segment  | Master Base Address | Range  | Master High Address |
|------------------------------|----------------|---------------|---------------------|--------|---------------------|
| Sysreg                       | S00_AXI        | S00_AXI_reg   | 0xA001_0000        | 64K    | 0xA001_FFFF        |
| AXI CDMA S_AXI_LITE          | S_AXI_LITE     | Reg           | 0xA000_0000        | 64K    | 0xA000_FFFF        |
| Output Data (bram_ctrl)      | S_AXI          | Mem0          | 0xE800_0000        | 256K   | 0xD003_FFFF        |
| h_data (bram_ctrl)           | S_AXI          | Mem0          | 0xE000_0000        | 1M     | 0xE00F_FFFF        |
| node_info (bram_ctrl)        | S_AXI          | Mem0          | 0xE200_0000        | 64K    | 0xE200_FFFF        |
| wgt_a (bram_ctrl)            | S_AXI          | Mem0          | 0xE400_0000        | 128K   | 0xE401_FFFF        |
| DDR Low                      | S_AXI_HP0_FPD  | HP0_DDR_LOW   | 0x0000_0000        | 2G     | 0x7FFF_FFFF        |
| QSPI                         | S_AXI_HP0_FPD  | HP0_QSPI      | 0xC100_0000        | 16M    | 0xC1FF_FFFF        |




In [1]:
import os
import sys
import time
import pynq
from pynq import Overlay, allocate, MMIO
import numpy as np
from pynq_cdma import CDMA
from pprint import pprint
from prettytable import PrettyTable

In [2]:
# DATASET = "Cora"
DATASET = "CiteSeer"

In [3]:
dataset = DATASET.lower()
os.environ["DATASET"] = DATASET
os.environ["GATHOME"] = '/root/GAT_FPGA/gat_v2'
os.environ["DATASET_PATH"] = f'/root/GAT_FPGA/gat_v2/data/{dataset}/'
gat_main_path = os.getenv("GATHOME")
gat_dataset_path = os.getenv("DATASET_PATH")

In [4]:
sys.path.append(os.path.abspath("/root/GAT_FPGA/misc"))
from log import log
from LoadData import LoadData
from BRAM import BRAM

from LoadData import binary_to_decimal, decimal_to_binary
from Helper import *
from Validate import validate_input, validate_output
from Accelerator import Accelerator
from Quantization import *
print("DONE")

DONE


In [5]:
print(pynq.ps.Clocks.fclk0_mhz)

214.283571


# Select Dataset

In [6]:
GlobalConfiguration

{'GAT': {'hiddenChannel': 16, 'head': 1},
 'model': {'savePath': '/root/GAT_FPGA/model/citeseer_model.pth',
  'scaleMin': -127,
  'scaleMax': 127},
 'dataset': {'root': 'data/Planetoid',
  'name': 'CiteSeer',
  'normalization': False}}

In [7]:
layer_1 = 0
layer_2 = 1
full_layer = 2

In [8]:
if DATASET == "Cora":
    H_DATA_DEPTH_LAYER1, WEIGHT_DEPTH_LAYER1, NODE_INFO_DEPTH, FEAT_DEPTH_LAYER1 = 242101, 23086, 13264, 43328
    H_DATA_DEPTH_LAYER2, WEIGHT_DEPTH_LAYER2, FEAT_DEPTH_LAYER2 = 212224, 126, 18956

elif DATASET == "CiteSeer":
    H_DATA_DEPTH_LAYER1, WEIGHT_DEPTH_LAYER1, NODE_INFO_DEPTH, FEAT_DEPTH_LAYER1 = 399089, 59388, 12383, 52464
    H_DATA_DEPTH_LAYER2, WEIGHT_DEPTH_LAYER2, FEAT_DEPTH_LAYER2 = 198128, 108, 19674

In [9]:
h_data_bram    = BRAM(0xE0000000, H_DATA_DEPTH_LAYER1)
node_info_bram = BRAM(0xE2000000, NODE_INFO_DEPTH)
weight_bram    = BRAM(0xE4000000, WEIGHT_DEPTH_LAYER1)
subgraph_index_bram = BRAM(0xE6000000, NODE_INFO_DEPTH)
feat_out_bram  = BRAM(0xE8000000, FEAT_DEPTH_LAYER2)

BramLayer1 = {
    "h_data"    : h_data_bram,
    "node_info" : node_info_bram,
    "weight"    : weight_bram,
    "subgraph_index": subgraph_index_bram,
    "feat_out"  : feat_out_bram
}

In [10]:
h_data_bram2  = BRAM(0xE0000000, H_DATA_DEPTH_LAYER2)
weight_bram2  = BRAM(0xE4000000, WEIGHT_DEPTH_LAYER2)
feat_out_bram = BRAM(0xE8000000, FEAT_DEPTH_LAYER2)

BramLayer2 = {
    "h_data"   : h_data_bram2,
    "weight"   : weight_bram2,
    "feat_out" : feat_out_bram
}

In [11]:
sysreg_ip = {
    "BASE_ADDR": 0x00A0010000,
    "RANGE"    : 64*1024
}

REG = {
    "gat_layer"          : 0,
    "gat_load_done"      : 4,
    "wgt_load_done"      : 4,
    "h_data_load_done"   : 8,
    "node_info_load_done": 12,
    "gat_ready"          : 16,
    "i_gat_debug_1"      : 20,
    "i_gat_debug_2"      : 24,
    "i_gat_debug_3"      : 28,
}

## Prepare Data

In [12]:
# Software
data_loader_instance = DatasetLoaderV2()
gat_instance = GATV2(data_loader_instance)
model_instance = BuildModelV2(gat_instance)

Model parameters loaded from /root/GAT_FPGA/model/citeseer_model.pth


In [13]:
overlay_gat = Overlay(f"{gat_dataset_path}hw/design_gat_wrapper.bit")

## Layer 1

In [14]:
accelerator = Accelerator(overlay_gat, sysreg_ip, BramLayer1)

In [15]:
accelerator.prepare_data(full_layer)
accelerator.transfer(full_layer)

[LoadData] : reading /root/GAT_FPGA/gat_v2/data/citeseer//layer_1/input/h_data.txt
[LoadData] : reading /root/GAT_FPGA/gat_v2/data/citeseer//layer_1/input/node_info.txt
[LoadData] : reading /root/GAT_FPGA/gat_v2/data/citeseer//layer_1/input/weight.txt
[LoadData] : reading /root/GAT_FPGA/gat_v2/data/citeseer//layer_1/input/subgraph_index.txt

[Accelerator] :  Transfering Time = 10.509 ms
[Accelerator] :  Execution Time = 3.58 ms

DONE


In [16]:
print(accelerator.result_final_layer[:20])

[80.0, 192.0, 0.0, 494.0, 0.0, 61.0, 1023.8333435058594, 3016.3333282470703, 1331.0, 0.0, 0.0, 611.4999847412109, 0.0, 0.0, 639.0, 102.0, 0.0, 1780.0, 0.0, 86.66667175292969]


## Layer 2

In [17]:
validate_output(accelerator.result_final_layer, f"{gat_dataset_path}/layer_2/", report_status="NONE")

======================= LAYER 2 VALIDATION =======================

📊 [SUMMARY] → INFO: 15379 	 WARNING: 2953 	 ERROR: 1342



## Result

In [18]:
handle_classification(accelerator.result_final_layer, data_loader_instance, 2)

torch.Size([3327, 3703])

===================== FEATURE CLASSIFICATION =====================
- Dataset : CiteSeer - v2.0
- Golden  : tensor([3, 1, 5,  ..., 3, 1, 5])
- DUT     : tensor([3, 1, 5,  ..., 2, 1, 5])
 => Accuracy = 66.3 % (663 / 1000)


### Debugger

In [19]:
format_print(accelerator.sysreg.read(REG["i_gat_debug_3"]))

Last modified: 14:04 - 22/04


In [20]:
# import matplotlib.pyplot as plt
# from sklearn.manifold import TSNE
# import numpy as np